IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M6\FAULT_M6_5


In [3]:
guasto=1

VOLO m6 FAULT 5%

In [4]:
rcou_m6_fault5 = pd.read_csv("RCOU.csv")
rcou_m6_fault5 = rcou_m6_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m6_fault5 = rcou_m6_fault5[((rcou_m6_fault5['C9'] >= potenza) & (rcou_m6_fault5['C10'] >= potenza) & (rcou_m6_fault5['C11'] >= potenza) & (rcou_m6_fault5['C12'] >= potenza) & (rcou_m6_fault5['C13'] >= potenza) & (rcou_m6_fault5['C14']>= potenza))]
rcou_m6_fault5=rcou_m6_fault5.reset_index(drop=True)
display(rcou_m6_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,67658101,1396,1412,1424,1383,1411,1397
1,67668340,1449,1369,1409,1412,1443,1376
2,67681963,1456,1378,1418,1418,1452,1382
3,67702125,1476,1381,1458,1401,1497,1354
4,67712889,1372,1494,1500,1365,1438,1435
...,...,...,...,...,...,...,...
4875,125263513,1623,1385,1616,1395,1566,1459
4876,125280870,1598,1400,1542,1470,1471,1540
4877,125291537,1570,1417,1577,1408,1491,1505
4878,125301650,1542,1426,1565,1397,1463,1509


In [5]:
min=rcou_m6_fault5['TimeUS'][0]
max=rcou_m6_fault5['TimeUS'][len(rcou_m6_fault5)-1]
print(max)
print(min)

125327720
67658101


In [6]:
xkf1_m6_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m6_fault5 = xkf1_m6_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m6_fault5 = xkf1_m6_fault5[((xkf1_m6_fault5['TimeUS'] >= min) & (xkf1_m6_fault5['TimeUS'] <= max	))]
xkf1_m6_fault5 = xkf1_m6_fault5.reset_index(drop=True)
print(xkf1_m6_fault5)

         TimeUS  Roll  Pitch    Yaw
0      67666069  0.54  -1.23  28.83
1      67679934  0.55  -1.25  28.82
2      67689697  0.66  -1.31  28.83
3      67700022  0.69  -1.36  28.82
4      67710276  0.63  -1.35  28.83
...         ...   ...    ...    ...
5214  125278300 -2.21   2.07  12.35
5215  125289227 -2.17   1.91  12.25
5216  125299633 -2.14   1.77  12.11
5217  125309609 -2.09   1.60  11.97
5218  125325522 -1.98   1.23  11.72

[5219 rows x 4 columns]


In [7]:
att_m6_fault5 = pd.read_csv("ATT.csv")
att_m6_fault5 = att_m6_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m6_fault5 = att_m6_fault5[((att_m6_fault5['TimeUS'] >= min) & (att_m6_fault5['TimeUS'] <= max	))]
att_m6_fault5=att_m6_fault5.reset_index(drop=True)
print(att_m6_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      67667560     0.11  0.54      0.42  -1.23   28.71  28.83
1      67681139     0.10  0.55      0.43  -1.25   28.71  28.82
2      67691201     0.09  0.66      0.45  -1.31   28.71  28.83
3      67701334     0.08  0.69      0.47  -1.36   28.71  28.82
4      67712253     0.07  0.63      0.48  -1.35   28.71  28.83
...         ...      ...   ...       ...    ...     ...    ...
5214  125280024    -1.27 -2.21      1.24   2.07   12.32  12.35
5215  125290917    -1.27 -2.17      1.24   1.91   12.32  12.25
5216  125300929    -1.29 -2.14      1.25   1.77   12.32  12.11
5217  125311929    -1.32 -2.09      1.28   1.60   12.32  11.97
5218  125327033    -1.36 -1.98      1.30   1.23   12.32  11.72

[5219 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m6_fault5)>len(xkf1_m6_fault5):
    to_add=att_m6_fault5[len(xkf1_m6_fault5):]
    att_m6_fault5=att_m6_fault5[:len(xkf1_m6_fault5)]

if len(xkf1_m6_fault5)>len(att_m6_fault5):
    to_add=xkf1_m6_fault5[len(att_m6_fault5):]

for idx,i in enumerate(att_m6_fault5['Roll']):
    if(xkf1_m6_fault5['Roll'][idx] != i):
        att_m6_fault5['Roll'][idx] = ((att_m6_fault5['Roll'][idx] + xkf1_m6_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m6_fault5['Pitch']):
    if(xkf1_m6_fault5['Pitch'][idx] != i):
        att_m6_fault5['Pitch'][idx] = ((att_m6_fault5['Pitch'][idx] + xkf1_m6_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m6_fault5['Yaw']):
    if(xkf1_m6_fault5['Yaw'][idx] != i):
        att_m6_fault5['Yaw'][idx] = ((att_m6_fault5['Yaw'][idx] + xkf1_m6_fault5['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m6_fault5=pd.concat([att_m6_fault5,to_add])

print(att_m6_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      67667560     0.11  0.54      0.42  -1.23   28.71  28.83
1      67681139     0.10  0.55      0.43  -1.25   28.71  28.82
2      67691201     0.09  0.66      0.45  -1.31   28.71  28.83
3      67701334     0.08  0.69      0.47  -1.36   28.71  28.82
4      67712253     0.07  0.63      0.48  -1.35   28.71  28.83
...         ...      ...   ...       ...    ...     ...    ...
5214  125280024    -1.27 -2.21      1.24   2.07   12.32  12.35
5215  125290917    -1.27 -2.17      1.24   1.91   12.32  12.25
5216  125300929    -1.29 -2.14      1.25   1.77   12.32  12.11
5217  125311929    -1.32 -2.09      1.28   1.60   12.32  11.97
5218  125327033    -1.36 -1.98      1.30   1.23   12.32  11.72

[5219 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m6_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m6_fault5 = esc_0_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m6_fault5 = esc_0_m6_fault5[((esc_0_m6_fault5['TimeUS'] >= min) & (esc_0_m6_fault5['TimeUS'] <= max))]
esc_0_m6_fault5=esc_0_m6_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m6_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m6_fault5 = esc_1_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m6_fault5 = esc_1_m6_fault5[((esc_1_m6_fault5['TimeUS'] >= min) & (esc_1_m6_fault5['TimeUS'] <= max))]
esc_1_m6_fault5=esc_1_m6_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m6_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m6_fault5 = esc_2_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m6_fault5 = esc_2_m6_fault5[((esc_2_m6_fault5['TimeUS'] >= min) & (esc_2_m6_fault5['TimeUS'] <= max))]
esc_2_m6_fault5=esc_2_m6_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m6_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m6_fault5 = esc_3_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m6_fault5 = esc_3_m6_fault5[((esc_3_m6_fault5['TimeUS'] >= min) & (esc_3_m6_fault5['TimeUS'] <= max))]
esc_3_m6_fault5=esc_3_m6_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m6_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m6_fault5 = esc_4_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m6_fault5 = esc_4_m6_fault5[((esc_4_m6_fault5['TimeUS'] >= min) & (esc_4_m6_fault5['TimeUS'] <= max))]
esc_4_m6_fault5=esc_4_m6_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m6_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m6_fault5 = esc_5_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m6_fault5 = esc_5_m6_fault5[((esc_5_m6_fault5['TimeUS'] >= min) & (esc_5_m6_fault5['TimeUS'] <= max))]
esc_5_m6_fault5=esc_5_m6_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m6_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m6_fault5 = imu_0_m6_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m6_fault5 = imu_0_m6_fault5[((imu_0_m6_fault5['TimeUS'] >= min) & (imu_0_m6_fault5['TimeUS'] <= max))]

imu_1_m6_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m6_fault5 = imu_1_m6_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m6_fault5 = imu_1_m6_fault5[((imu_1_m6_fault5['TimeUS'] >= min) & (imu_1_m6_fault5['TimeUS'] <= max))]

imu_2_m6_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m6_fault5 = imu_2_m6_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m6_fault5 = imu_2_m6_fault5[((imu_2_m6_fault5['TimeUS'] >= min) & (imu_2_m6_fault5['TimeUS'] <= max))]

imu_m6_fault5 = pd.concat((imu_0_m6_fault5, imu_1_m6_fault5, imu_2_m6_fault5))
imu_m6_fault5=imu_m6_fault5.groupby(imu_m6_fault5.TimeUS, as_index=False).mean()

display(imu_m6_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,67659762,-0.138047,-0.038018,0.001737,0.033579,0.086641,-9.665883
1,67662879,-0.011009,-0.058367,-0.001887,0.068102,0.299002,-9.464789
2,67665259,0.100030,-0.062974,-0.002615,0.024549,0.329164,-9.613195
3,67667399,0.097606,-0.021730,0.007218,-0.066825,0.120736,-9.898083
4,67670078,-0.066935,0.001945,0.007067,-0.104688,0.004512,-10.003252
...,...,...,...,...,...,...,...
19887,125308469,0.163667,-0.395403,-0.270902,1.135757,1.445938,-12.786887
19888,125311848,0.078180,-0.325693,-0.265427,0.796094,1.550297,-12.949453
19889,125314499,0.040967,-0.191538,-0.240225,0.893079,1.877303,-12.723017
19890,125316963,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m6_fault5 = pd.merge_ordered(imu_m6_fault5,att_m6_fault5)
m6_fault5=m6_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_0_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_1_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_2_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_3_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_4_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_5_m6_fault5)
m6_fault5=m6_fault5.fillna(method="ffill").fillna(method="bfill")
m6_fault5 = pd.merge_ordered(m6_fault5, rcou_m6_fault5)
m6_fault5=m6_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m6_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m6_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m6_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m6_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m6_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m6_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m6_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m6_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

5219
5710
6194
6682
7170
7660
8154


In [13]:
m6_fault5 = m6_fault5[~m6_fault5.TimeUS.isin(index_to_remove)]
m6_fault5["TimeUS"] = m6_fault5["TimeUS"] - m6_fault5.iloc[0]["TimeUS"]
m6_fault5["TimeUS"] = m6_fault5["TimeUS"].astype(int)
m6_fault5["Guasto"] = guasto

In [14]:
from datetime import timedelta
m6_fault5=m6_fault5.reset_index(drop=True)
m6_fault5['TimeUS'] = pd.to_datetime(m6_fault5['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m6_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.138047,-0.038018,0.001737,0.033579,0.086641,-9.665883,0.11,0.54,0.42,...,2.67,3609.0,1.96,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1
1,00:00:00.003117,-0.011009,-0.058367,-0.001887,0.068102,0.299002,-9.464789,0.11,0.54,0.42,...,2.67,3609.0,1.96,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1
2,00:00:00.005497,0.100030,-0.062974,-0.002615,0.024549,0.329164,-9.613195,0.11,0.54,0.42,...,2.67,3609.0,1.96,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1
3,00:00:00.007637,0.097606,-0.021730,0.007218,-0.066825,0.120736,-9.898083,0.11,0.54,0.42,...,2.67,3609.0,1.96,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1
4,00:00:00.010316,-0.066935,0.001945,0.007067,-0.104688,0.004512,-10.003252,0.11,0.54,0.42,...,2.67,3609.0,1.96,1449.0,1369.0,1409.0,1412.0,1443.0,1376.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19887,00:00:57.648707,0.163667,-0.395403,-0.270902,1.135757,1.445938,-12.786887,-1.29,-2.14,1.25,...,2.06,5898.0,2.77,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1
19888,00:00:57.652086,0.078180,-0.325693,-0.265427,0.796094,1.550297,-12.949453,-1.29,-2.14,1.25,...,2.06,5898.0,2.77,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1
19889,00:00:57.654737,0.040967,-0.191538,-0.240225,0.893079,1.877303,-12.723017,-1.32,-2.09,1.28,...,2.06,5898.0,2.77,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1
19890,00:00:57.657201,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253,-1.32,-2.09,1.28,...,2.06,5898.0,2.77,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m6_fault5)/350)):
        V1=m6_fault5[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28232\543051110.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28232\543051110.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.093381,0.037876,-0.371872,0.215611,57.0,56.0,6.819656,2.546067,0.052726,...,8693.240121,3.141652,0.578078,-0.923406,3.232091,137.0,136.0,49.102992,32.377058,1
1,1.0,-0.094341,0.038093,-0.384505,0.216529,59.0,58.0,7.467662,2.592531,0.052629,...,8438.647779,3.143580,0.577738,-0.923768,3.233912,139.0,138.0,49.952662,32.208387,1
2,2.0,-0.095220,0.038257,-0.397279,0.217291,61.0,60.0,8.309405,2.424888,0.052336,...,8174.046816,3.145496,0.577393,-0.924047,3.235723,135.0,134.0,50.774488,32.014294,1
3,3.0,-0.095987,0.038357,-0.408481,0.217858,60.0,59.0,8.226138,2.349315,0.051866,...,7899.417307,3.147401,0.577043,-0.924246,3.237522,139.0,138.0,51.568112,31.795052,1
4,4.0,-0.096790,0.038478,-0.420129,0.218490,58.0,57.0,6.781630,2.460870,0.051580,...,7614.755899,3.149296,0.576687,-0.924365,3.239310,137.0,136.0,52.333204,31.550979,1
5,5.0,-0.097624,0.038617,-0.431948,0.219178,60.0,59.0,6.792733,2.544485,0.051442,...,7640.332731,3.151180,0.576326,-0.924405,3.241086,141.0,140.0,53.069463,31.282435,1
6,6.0,-0.098474,0.038766,-0.443682,0.219897,59.0,58.0,7.561007,2.544665,0.051169,...,7838.716379,3.153053,0.575960,-0.924369,3.242852,123.0,122.0,53.776616,30.989824,1
7,7.0,-0.099201,0.038847,-0.453420,0.220407,63.0,62.0,8.405169,2.307198,0.050919,...,8034.497926,3.154916,0.575589,-0.924256,3.244607,141.0,140.0,54.454416,30.673593,1
8,8.0,-0.099853,0.038891,-0.461156,0.220802,55.0,54.0,8.157120,2.420232,0.050684,...,8227.334736,3.156769,0.575214,-0.924069,3.246352,137.0,136.0,55.102645,30.334231,1
9,9.0,-0.100684,0.039031,-0.472169,0.221495,67.0,66.0,6.660049,2.437236,0.050403,...,8416.909361,3.158611,0.574833,-0.923808,3.248086,141.0,140.0,55.721110,29.972266,1


In [18]:
path_file = path_file.replace(r"M6\FAULT_M6_5", "")
os.chdir(path_file)
df_merged.to_csv('m6_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
